In [1]:
whoami; date; hostname; pwd;

ytanigaw
Thu May  9 15:33:30 PDT 2019
sh-102-07.int
/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/ukbb-tools/04_gwas/ref_alt


In [2]:
ml load bedtools

#### reference sequence file

In [3]:
cat /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa | head -n2

>chr1
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
cat: write error: Broken pipe


In [4]:
cat /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa \
| egrep '^>' | grep -v '_'

>chr1
>chr2
>chr3
>chr4
>chr5
>chr6
>chr7
>chrX
>chr8
>chr9
>chr10
>chr11
>chr12
>chr13
>chr14
>chr15
>chr16
>chr17
>chr18
>chr20
>chrY
>chr19
>chr22
>chr21
>chrM


# Array

### Compare UKBB's A1/A2 with the reference sequence

#### tl;dr: UKBB A1 column (5-th column in the bim file) matches with the reference sequence

We will take chr22 as an example case first. This is the bim file from the UKBB.

In [3]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_chr22_v2.bim | head -n10

22	rs62224618	0	16057417	C	T
22	Affx-52233492	0	16287538	G	C
22	rs200923174	0	16287557	C	G
22	Affx-80289661	0	16287585	AC	A
22	Affx-80289662	0	16287663	TC	T
22	Affx-52336937	0	16287779	CT	C
22	rs138257042	0	16449075	G	A
22	rs116911124	0	16495833	C	A
22	rs117578132	0	16595552	C	A
22	rs117982183	0	16648658	C	T
cat: write error: Broken pipe


We query the reference fasta file with `bedtools getfasta` command, which requires UCSC bed file as input so I converted the PLINK bim file to UCSC bed file using awk. 
Specifically, PLINK bim is on 1-based coordinate and UCSC bed is on 0-based semi-intervals. 


In the UCSC bed file, we generated the name in the following format:
`<variant ID>:<chromosome>:<position (1-based index)>:<A1 (5-th column in bim file)>:<A2 (6-th column in bim file)>`

We pipe this UCSC bed file into `bedtools getfasta` command to get the reference sequence, which is shown at the last column of the input


In [4]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_chr22_v2.bim | head -n10 \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($5), $2 sep chr $1 sep $4 sep $5 sep $6}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa -bed /dev/stdin -bedOut

cat: write error: Broken pipe
chr22	16057416	16057417	rs62224618:chr22:16057417:C:T	C
chr22	16287537	16287538	Affx-52233492:chr22:16287538:G:C	G
chr22	16287556	16287557	rs200923174:chr22:16287557:C:G	C
chr22	16287584	16287586	Affx-80289661:chr22:16287585:AC:A	AC
chr22	16287662	16287664	Affx-80289662:chr22:16287663:TC:T	TC
chr22	16287778	16287780	Affx-52336937:chr22:16287779:CT:C	CT
chr22	16449074	16449075	rs138257042:chr22:16449075:G:A	G
chr22	16495832	16495833	rs116911124:chr22:16495833:C:A	c
chr22	16595551	16595552	rs117578132:chr22:16595552:C:A	c
chr22	16648657	16648658	rs117982183:chr22:16648658:C:T	c


As you can see above, the reference sequence matches with the A1 column in the bim file.

We now test all autosomes to see if there are mismatches.

In [8]:
cat /oak/stanford/groups/mrivas/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_unpatched_ukbb.bim \
| sed -e 's/^23/X/g' | sed -e 's/^24/Y/g' | sed -e 's/^25/X/g' | sed -e 's/^26/M/g' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($5), $2 sep chr $1 sep $4 sep $5 sep $6}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 != toupper($9)'



chr4	7435304	7435305	Affx-24919403	chr4	7435305	T	C	A
chr7	150896002	150896003	Affx-29977087	chr7	150896003	T	G	C
chr12	53011954	53011955	Affx-8165091	chr12	53011955	T	C	G
chr21	46354974	46354975	Affx-18140622	chr21	46354975	A	G	C
chr22	39218763	39218764	Affx-19589717	chr22	39218764	T	C	G
chrM	72	73	Affx-79504644	chrM	73	A	G	G
chrM	149	150	Affx-52321525	chrM	150	C	T	T
chrM	262	263	Affx-34461957	chrM	263	A	G	G
chrM	455	456	Affx-79381653	chrM	456	C	T	T
chrM	546	547	Affx-89025725	chrM	547	A	T	G
chrM	708	709	Affx-34462196	chrM	709	G	A	C
chrM	749	750	Affx-79381656	chrM	750	A	G	G
chrM	768	769	Affx-34462230	chrM	769	G	A	C
chrM	826	827	Affx-89025774	chrM	827	A	G	T
chrM	979	980	Affx-91439598	chrM	980	T	C	A
chrM	1017	1018	Affx-89025772	chrM	1018	G	A	T
chrM	1242	1243	Affx-34461684	chrM	1243	T	C	C
chrM	1390	1391	Affx-89025736	chrM	1391	T	C	G
chrM	1405	1406	Affx-89025696	chrM	1406	T	C	A
chrM	1437	1438	Affx-34461788	chrM	1438	A	G	C
chrM	1718	1719	Affx-79381658	chrM	1719	G	A	T
chrM	1720	1721	Affx-794

In [10]:
for c in $(seq 1 22) X Y XY MT ; do cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_chr${c}_v2.bim ; done | wc


 805426 4832556 23115529


In [9]:
wc $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_unpatched_ukbb.bim

  805426  4832556 23135077 /oak/stanford/groups/mrivas/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_unpatched_ukbb.bim


In [13]:
for c in $(seq 1 22) X Y XY MT ; do cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_chr${c}_v2.bim ; done | md5sum



2af900ed28f26de3c51796f2d4b04cfe  -


In [14]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_unpatched_ukbb.bim \
| sed -e 's/^23/X/g' | sed -e 's/^24/Y/g' | sed -e 's/^25/XY/g' | sed -e 's/^26/MT/g' \
| md5sum

2af900ed28f26de3c51796f2d4b04cfe  -


In [15]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_unpatched_ukbb.bim | awk '$1 <= 22' | wc -l

784256


There are 5/784256 exceptions in autosomes, which can be due to minor version difference of the reference genome (we used GRCh37.p13, but UKBB may have used other minor versions, such as GRCh37.p12)

We also found that there are bunch of differences in chrM ...

### ukb24983_cal_cALL_v2_hg19

We prepare a fixed version as follows:

```
# 0. namespace
o1="ukb24983_cal_cALL_v2_unpatched_ukbb"
o2="ukb24983_cal_cALL_v2_hg19"

# 1. combine split chr bed/bim/fam files
plink --merge-list chr_bfile_list.txt --make-bed --keep-allele-order \
      --out $o1 \
      --memory 60000

# 1b. make pgen
plink2 --bfile $o1 --keep-allele-order --make-pgen \
       --out ${o1}.p \
       --memory 60000

# 1c. move files back
for end in "pgen" "psam" "pvar"; do
    mv ${o1}.p.${end} ${o1}.${end}
done

# 2. flip alleles (ukbb-ref is backwards relative to hg19)
plink2 --pfile $o1 --make-bed --ref-allele ${o1}.bim 5 2 \
       --out $o2 \
       --memory 60000

plink2 --pfile $o1 --make-pgen --ref-allele ${o1}.bim 5 2 \
       --out ${o2}.p \
       --memory 60000

for end in "pgen" "psam" "pvar"; do
    mv ${o2}.p.${end} ${o2}.${end}
done
```

In [20]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_hg19.bim \
| awk -v OFS='\t' '{print $1, $2, $3, $4, $6, $5}' | md5sum

2af900ed28f26de3c51796f2d4b04cfe  -


In [21]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_unpatched_ukbb.bim \
| sed -e 's/^23/X/g' | sed -e 's/^24/Y/g' | sed -e 's/^25/XY/g' | sed -e 's/^26/MT/g' \
| md5sum

2af900ed28f26de3c51796f2d4b04cfe  -


As you can see below, the reference sequence (UCSC hg19) matches with the A2 column in the v2_hg19 bim file.


In [22]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_cALL_v2_hg19.bim \
| sed -e 's/^XY/X/g' | sed -e 's/^MT/M/g' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($6), $2 sep chr $1 sep $4 sep $6 sep $5}' \
| grep -v '^chrM' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 != toupper($9)'


chr4	7435304	7435305	Affx-24919403	chr4	7435305	T	C	A
chr7	150896002	150896003	Affx-29977087	chr7	150896003	T	G	C
chr12	53011954	53011955	Affx-8165091	chr12	53011955	T	C	G
chr21	46354974	46354975	Affx-18140622	chr21	46354975	A	G	C
chr22	39218763	39218764	Affx-19589717	chr22	39218764	T	C	G


### Variant annotation table

`/oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.tsv.gz`

This has autosome variants only


In [23]:
zcat /oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.tsv.gz \
| cut -f1-5 | head -n3


CHROM	POS	REF	ALT	ID
1	723307	C	G	rs28659788
1	727841	G	A	rs116587930
cut: write error: Broken pipe


In [24]:
zcat /oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.tsv.gz \
| cut -f1-5 \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '(NR>1){print chr $1, $2-1, $2-1+length($3), $5 sep chr $1 sep $2 sep $3 sep $4}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 != toupper($9)'



chr12	53011954	53011955	Affx-8165091	chr12	53011955	T	C	G
chr21	46354974	46354975	Affx-18140622	chr21	46354975	A	G	C
chr22	39218763	39218764	Affx-19589717	chr22	39218764	T	C	G
chr4	7435304	7435305	Affx-24919403	chr4	7435305	T	C	A
chr7	150896002	150896003	Affx-29977087	chr7	150896003	T	G	C


### example summary statistics file

In [32]:
zcat /oak/stanford/groups/mrivas/ukbb24983/cal/gwas/9796/24611/white_british/ukb24983_v2.INI4022.genotyped.glm.linear.gz \
| cut -f 1-6 \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '(NR>1){print chr $1, $2-1, $2-1+length($4), $3 sep chr $1 sep $2 sep $3 sep $4 sep $5 sep $6}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$8 != toupper($11) || $9 != $10'


chr4	7435304	7435305	Affx-24919403	chr4	7435305	Affx-24919403	T	C	C	A
chr7	150896002	150896003	Affx-29977087	chr7	150896003	Affx-29977087	T	G	G	C
chr12	53011954	53011955	Affx-8165091	chr12	53011955	Affx-8165091	T	C	C	G
chr21	46354974	46354975	Affx-18140622	chr21	46354975	Affx-18140622	A	G	G	C
chr22	39218763	39218764	Affx-19589717	chr22	39218764	Affx-19589717	T	C	C	G


### liftOver: ukb24983_cal_cALL_v2_1.hg38.bim

Using the liftOver, a UCSC genome browser tool, we mapped the bim file to hg38.
Here are the summary of the liftOver and REF/ALT check status.


| _ | LiftOver and REF/ALT check status | Number of variants |
| --------- | -------------------------------------------------------- | ------ |
| (1) | Successfully mapped and REF (A2) matches with GRCh38.p13 | 781,659 |
| (2) | Successfully mapped but ALT (A1) matches with GRCh38.p13 |   1,343 |
| (3) | Successfully mapped but REF/ALT (A1/A2) does not match GRCh38.p13 |   960 |
| (4) | Mapped to somewhere but the coordinates are invalid | 1 |
| (5) | Unmapped (UCSC liftOver failed to map) | 293 |
|  _  | TOTAL | 784,256 |

In [24]:
wc -l ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim

784256 ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim


In [23]:
cat ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim \
| awk '$4 != 0' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($6), $2 sep chr $1 sep $4 sep $6 sep $5}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg38/hg38.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 == toupper($9)' | wc -l


Feature (chr19:101658107-101658108) beyond the length of chr19 size (58617616 bp).  Skipping.
781659


In [15]:
cat ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim \
| awk '$4 != 0' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($6), $2 sep chr $1 sep $4 sep $6 sep $5}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg38/hg38.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 != toupper($9)' | wc -l

Feature (chr19:101658107-101658108) beyond the length of chr19 size (58617616 bp).  Skipping.
2303


In [20]:
cat ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim \
| awk '$4 != 0' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($6), $2 sep chr $1 sep $4 sep $6 sep $5}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg38/hg38.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 != toupper($9) && $8 == toupper($9)' | wc -l


Feature (chr19:101658107-101658108) beyond the length of chr19 size (58617616 bp).  Skipping.
1343


In [22]:
cat ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim \
| awk '$4 != 0' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($6), $2 sep chr $1 sep $4 sep $6 sep $5}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg38/hg38.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 != toupper($9) && $8 != toupper($9)' | wc -l


Feature (chr19:101658107-101658108) beyond the length of chr19 size (58617616 bp).  Skipping.
960


In [21]:
cat ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim \
| awk '$4 != 0' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($6), $2 sep chr $1 sep $4 sep $6 sep $5}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg38/hg38.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 != toupper($9) && $8 != toupper($9)' | head


chr1	12719556	12719558	Affx-52321985	chr1	12719557	AT	A	AC
chr1	13482277	13482278	Affx-4972073	chr1	13482278	C	T	G
chr1	143956444	143956445	rs78105048	chr1	143956445	T	C	A
chr1	143943802	143943803	rs146627141	chr1	143943803	T	C	A
chr1	143938433	143938434	rs144715521	chr1	143938434	T	C	G
chr1	143904543	143904544	rs115198068	chr1	143904544	G	A	C
chr1	143883551	143883552	rs149651017	chr1	143883552	T	G	A
chr1	143880556	143880557	rs149452988	chr1	143880557	G	A	C
chr1	143876237	143876238	rs200153581	chr1	143876238	C	T	G
chr1	82102	82103	rs147887611	chr1	82103	G	A	t
awk: cmd. line:1: (FILENAME=- FNR=33733) fatal: print to "standard output" failed (Broken pipe)
tr: write error: Broken pipe
tr: write error
Feature (chr19:101658107-101658108) beyond the length of chr19 size (58617616 bp).  Skipping.


In [25]:
md5sum ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim

34917faa1aa42af954d7fc7bc510c050  ../../09_liftOver/ukb24983_cal_cALL_v2_1.hg38.bim


# Exome

Their variant IDs for SNPs are `<chr>:<pos>:<ref>:<alt>` and the A2 column (6-th column in the bim file) matches with the reference sequence

In [10]:
cat $OAK/private_data/ukbb/24983/exome/pgen/spb/data/ukb_exm_spb.bim | tr ":" "\t" | awk '$4 != $9' | cut -f4 | sort | uniq -c

 587679 D
 261956 I


In [11]:
cat $OAK/private_data/ukbb/24983/exome/pgen/spb/data/ukb_exm_spb.bim \
| sed -e 's/^23/X/g' | sed -e 's/^24/Y/g' | sed -e 's/^25/XY/g' | sed -e 's/^26/MT/g' | sed -e 's/:/_/g' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($6), $2 sep chr $1 sep $4 sep $6 sep $5}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg38/hg38.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | sed -e 's/_/:/g' \
| awk '$7 != toupper($9)'


In [12]:
cat $OAK/private_data/ukbb/24983/exome/pgen/fe/data/ukb_exm_fe.bim | tr ":" "\t" | awk '$4 != $9' | cut -f4 | sort | uniq -c

 445109 D
 319025 I


In [13]:
cat $OAK/private_data/ukbb/24983/exome/pgen/fe/data/ukb_exm_fe.bim \
| sed -e 's/^23/X/g' | sed -e 's/^24/Y/g' | sed -e 's/^25/XY/g' | sed -e 's/^26/MT/g' | sed -e 's/:/_/g' \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($6), $2 sep chr $1 sep $4 sep $6 sep $5}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg38/hg38.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | sed -e 's/_/:/g' \
| awk '$7 != toupper($9)' | head
